## Next Steps:

## History
- 17.11.19: added Depot as bubble
- 17.11.19: 3M Performance now in 20% quantiles
- 16.11.19: initial version

In [165]:
import datetime
import pandas as pd

In [166]:
def floatconv(val):
    try:
        if val.strip():
            return float(val.replace('.','').replace(',','.'))
        else:
            return 0
    except ValueError as ve:
        print("VALUE NOT USABLE for floatconv: #{}#".format(val))

In [167]:
def percentconv(val):
    try:
        if '%' in val:
            return floatconv(val.replace('%', ''))
        else:
            return val
    except ValueError as ve:
        print("VALUE NOT USABLE for percentconv: #{}#".format(val))

In [168]:
intconv = lambda val: 0 if len(str(val)) < 2 else float(str(val).replace('.',''))
converter = {'Aktuell':floatconv, 'Wert in EUR':floatconv, 'Perf. 3 Monate':percentconv, \
             'Perf. 1 Jahr':percentconv, 'Perf. 3 Jahre':percentconv}

In [169]:
long2short = {'Amundi Index Solutio':'Nasdaq',
'Commerzbank AG DAX I':'CobaDax',
'ComStage 1 TecDAX UC':'ComsTecDx',
'Deutsche Telekom AG ':'DTE',
'DWS Aktien Strategie':'DWS AS',
'Fidelity Funds - Eur':'FidEur',
'Fidelity Funds - Glo':'FidGlTec',
'Fidelity Funds - Mal':'FidMal',
'Fidelity Funds - Nor':'FidNordic',
'iShares Automation &':'iShAuto',
'iShares TecDAX(DE)UC':'iShTecDax',
'Lang &#38; Schwarz A':'LS9AJF',
'LBBW Dividenden Stra':'LBBW',
'Nordea 1 Nordic Equi':'Nordea1',
'OEkoworld Klima - C ':'OekoKlima',
'OEkoWorld OEkoVision':'Oekovision',
'SAP SE Inhaber-Aktie':'SAP',
'SEB Nordic Small Cap':'NorSmallCap',
'WisdomTree Brent Cru':'Brent',
'Xtrackers MSCI World':'XtrMinVol',
'Xtrackers MSCI World':'XtrMSCIMomentum'}
def shortname(longname):
    return long2short[longname]

In [170]:
#filename = "musterdepot_Komplett_meineuebersicht_20191112_1043.csv"
filename = "musterdepot_Komplett_meineuebersicht.csv"
data = pd.read_csv(filename, sep=";", header=2, encoding="iso-8859-1", converters = converter, usecols=[0,1,4,19,20,21])
data['Wert']=data['Stück']*data['Aktuell']
data['Name'] = data['Bezeichnung'].apply(lambda x: shortname(x[0:20]))

In [171]:
# add missing values:
#Nasdaq Performance
data.loc[0,'Perf. 3 Jahre'] = 61
#TecDax Performance
data.loc[2,'Perf. 3 Jahre'] = 75
#A2ANH0 Performance
data.loc[9,'Perf. 3 Jahre'] = 55

In [172]:
def perf2String(val,quantiles):
    if val < quantiles[0.2]:
        return 'lowest'
    if val < quantiles[0.4]:
        return 'low'
    if val < quantiles[0.6]:
        return 'mid'
    if val < quantiles[0.8]:
        return 'high'
    return 'highest'

In [173]:
quantiles = data['Perf. 3 Monate'].quantile([0.2,0.4,0.6,0.8])
data['Perf3MString'] = data['Perf. 3 Monate'].apply(lambda x: perf2String(x,quantiles))

In [174]:
data['Value3MAgo'] = data['Wert']/(1+data['Perf. 3 Monate']/100)
data['Value1YAgo'] = data['Wert']/(1+data['Perf. 1 Jahr']/100)
data['Value3YAgo'] = data['Wert']/(1+data['Perf. 3 Jahre']/100)

valuetoday = data['Wert'].sum()
v3mago = data['Value3MAgo'].sum()
v1yago = data['Value1YAgo'].sum()
v3yago = data['Value3YAgo'].sum()
p3m = (valuetoday/v3mago-1)*100
p1y = (valuetoday/v1yago-1)*100
p3y = (valuetoday/v3yago-1)*100
print("Portfolio total value: {0:7.2f}, 3-month-performance: {1:3.2f}% , 1Y-perf: {2:3.2f}%, 3Y-perf: {3:3.2f}%"
      .format(valuetoday,p3m,p1y,p3y))

Portfolio total value: 42321.30, 3-month-performance: 8.23% , 1Y-perf: 18.74%, 3Y-perf: 44.40%


In [175]:
data

,Stück,Bezeichnung,Aktuell,Perf. 3 Monate,Perf. 1 Jahr,Perf. 3 Jahre,Wert,Name,Perf3MString,Value3MAgo,Value1YAgo,Value3YAgo
0,40,Amundi Index Solutions NASDAQ 100 UCITS ETF - ...,83.44,9.72,26.01,61,3337.6,Nasdaq,high,3041.924900,2648.678676,2073.04
1,100,Fidelity Funds - European Growth Fund - A EUR DIS,16.34,9.27,11.30,19.63,1634.0,FidEur,mid,1495.378420,1468.104223,1365.88
2,50,Fidelity Funds - Global Technology Fund - Y EU...,34.25,17.48,33.18,75,1712.5,FidGlTec,highest,1457.694927,1285.853732,978.571
3,50,Fidelity Funds - Nordic Fund - A SEK DIS,126.52,9.22,3.57,17.56,6326.0,FidNordic,mid,5791.979491,6107.946317,5381.08
4,220,iShares TecDAX(DE)UCITS ETF - EUR ACC,27.63,11.09,16.30,70.81,6078.6,iShTecDax,high,5471.779638,5226.655202,3558.69
5,50,OEkoworld Klima - C EUR ACC,79.58,2.49,22.35,50.86,3979.0,OekoKlima,lowest,3882.329983,3252.145484,2637.54
6,20,OEkoWorld OEkoVision Classic - C EUR ACC,190.34,2.77,12.79,28.64,3806.8,Oekovision,lowest,3704.193831,3375.121908,2959.27
7,60,SAP SE Inhaber-Aktien o.N.,123.08,13.29,32.84,55.97,7384.8,SAP,highest,6518.492365,5559.168925,4734.76
8,150,Xtrackers MSCI World Minimum Volatility UCITS ...,31.22,3.70,20.36,36.13,4683.0,XtrMSCIMomentum,low,4515.911283,3890.827517,3440.09
9,100,Xtrackers MSCI World Momentum UCITS ETF - 1C U...,33.79,4.85,19.52,55,3379.0,XtrMSCIMomentum,low,3222.699094,2827.141901,2180


In [176]:
def rd(val):
    return int(round(val))

In [177]:
#['Name','3JPerf','1J Perf','3M Perf','EUR'],
#['Nasdaq',32,22,'high',3214],

#lines with values:
values=""
for i, (index, row) in enumerate(data.iterrows()):
    values += "['"+row['Name']+"'," \
    +str(rd(row['Perf. 3 Jahre']))+"," \
    +str(rd(row['Perf. 1 Jahr']))+",'" \
    +row['Perf3MString']+"'," \
    +str(rd(row['Wert']))+"]," \
    +'\n'

#Depot line with 1y and 3y performance and 3m as part of the name (special color)    
values += "['Depot 3M:"+str(round(p3m,2))+"%'," \
+str(rd(p3y))+"," \
+str(rd(p1y))+"," \
+"'Depot','" \
+str(50000) \
+"']\n"

In [178]:
# read template, replace placeholder and write output file:
with open('portfolioPerformance_in.html','rt') as fin, open('portfolioPerformance.html','wt') as fout:
    for line in fin:
        if '#$0' in line:
            line = line.replace('#$0',values) 
        if '#$1' in line:
            today = datetime.date.today()
            line = line.replace('#$1',today.strftime('%d.%m.%Y'))
        if '#$2' in line:
            line = line.replace('#$2',str(quantiles.values))
        fout.write(line)